# Modeling Subsurface Dikes on the Moon

Weigang Liang <br/>


## 1. Background

The GRAIL Bouguer gravity gradient reveals quasi-rectangular anomalies on the near side that correlate well with the border of the PKT region, whereas in the topography the transition is far less defined. Those border structures have been interpreted as volcanically flooded rift valleys. On both the near and farsides, a large number of randomly oriented linear gravity anomalies (LGAs) have been identified with no surface expression in other datasets. These linear anomalies have been proposed to be dike-like structures that formed very early in lunar history, possibly during a period of global expansion. 

Bouguer gravity = free-air-gravity (full gravity minus gravity from a rough approximation of the planet as a sphere or ellipsoid (geoid)) - gravity-from-topography (gravity from just the surface elevation)

Figure below is from Andrews-Hanna J. C. et al. (2014) Nature, 514, 68-71

<img src="liang_figures/comparisons.png" style="width:45%">



## 2. Power Spectra


We first generate the localized Bouguer power spectra of the near-side and far-side anomalies, and compare them to the spectra of background areas. We use the GRAIL 1200-degree gravity model GRGM1200A and a LOLA topography model, and analyze the data using the Spherical Harmonic Tools (SHTOOLS) software package. In accordance with the surface density difference between the maria and the feldspathic highlands, we have generated separate Bouguer gravity models for the near and far sides assuming surface densities of 3100 and 2500 kg/m3, respectively.

<img src="liang_figures/near_far.png" style="width:45%">

### Far Spectra Result

Both the FN and FSP (in the northern farside and SPA rim, respectively) LGA spectra are 1-2 orders of magnitudes above the farside background power from degrees 80-200. The FN spectrum re-joins the background spectrum at the high degrees, but the FSP spectrum remains greater than the background spectrum. The South Pole-Aitken rim region shows significantly greater gravity gradient anomalies than the rest of the farside, so the spectrum of an arbitrary area in the rim is also calculated for comparison to FSP. The FSP LGA spectrum still shows significantly greater power than the rim background area, albeit less so. 

<img src="liang_figures/power_spectra.png" style="width:45%">


## 3. Dike Modeling with Metropolis-Hastings MCMC

We next model a dike that can account for the increase in power from the far-side background to the FN anomaly. The dike is modeled as an elliptically tapered rectangular prism of 200 km in length, which roughly corresponds to the length of the anomaly. We first use a grid-search to identify the optimal top depth, height, and width under a least-squared-sum difference optimization function that compares the real power spectrum and a synthetic spectrum combining the gravity of the modeled dike with the background variability. Then, we use the Metropolis-Hastings MCMC algorithm to search for convergence on those three parameters. We generate multiple chains that start from different inertial conditions and test if the intra- and inter-chain variance ratio "r" of all samples are close to 1.



In [ ]:
import pyshtools


import numpy as np
import scipy.io as io
import time
import copy
import sys
start_time = time.time()

def random_coeff(power):
    
    l = power.shape[0]
    
    coeff = np.zeros((2,l,l))
    
    for x in range(l):
        
        coeff_t = np.random.randn(2*x+1)*power[x]**0.5/(2*x+1)**0.5
        
        
        coeff[0,x,:x+1] = coeff_t[:x+1]
        coeff[1,x,:x] = coeff_t[x+1:]
    
#    power2 = pyshtools.spectralanalysis.spectrum(coeff)
        
    return coeff

from_matlab = {}
from_matlab = io.loadmat('dike_param.mat')
mtsef_og = np.squeeze(from_matlab['mtsef_og'])
mtseff0 = np.squeeze(from_matlab['mtseff0'])
tapersff0 = from_matlab['tapersff0']

#Function to calculate the gravity of a tapered rectangular prism
#td = top depth
#th = thickness
#xs = width
#div = number of rectangles that make up the ellipse

def forward_grav(td,th,xs,div):
    G = 6.67408e-11;
    drho = 500;
    grav = np.zeros((1002,2004));
        
    topo0 = 1738000*np.ones((1002,2004));
    
    lat = np.transpose(np.tile(np.linspace(0,180-180/1002,1002),(2004,1)))*np.pi/180;
    lon = np.tile(np.linspace(0,360-360/2004,2004),(1002,1))*np.pi/180;
    xp = topo0*np.sin(lat)*np.cos(lon);
    yp = topo0*np.sin(lat)*np.sin(lon);
    zp = topo0*np.cos(lat);
    
    latc = lat[120-1,940-1];
    lonc = lon[120-1,940-1];
    z_trans = np.array(((np.cos(lonc),np.sin(lonc),0),
        (-np.sin(lonc), np.cos(lonc), 0),
        (0, 0, 1)))
    y_trans = np.array(((np.cos(latc), 0, -np.sin(latc)),
        (0, 1, 0),
        (np.sin(latc), 0, np.cos(latc))))
    trans = np.matmul(y_trans,z_trans);
            
    xr = np.reshape(xp,(1,2004*1002)); yr = np.reshape(yp,(1,2004*1002)); zr = np.reshape(zp,(1,2004*1002));
    cr = np.matmul(trans,np.squeeze(np.stack((xr,yr,zr))));
    xp = np.reshape(cr[0,:],(1002,2004));
    yp = np.reshape(cr[1,:],(1002,2004));
    zp = np.reshape(cr[2,:],(1002,2004));
    
#        td = 43e3;
#        th = 9e3;
#        xs = 36e3;
    #ys = 130e3;
    ys = 200e3;
    x_s = np.zeros(div)
    
    if div%2==1:
        x_s[int((div-1)/2)] = xs
        x_s_part = (np.arange((div-1)/2)+1)*2*xs/div
        x_s[:int((div-1)/2)] = x_s_part
        x_s[int((div-1)/2+1):] = x_s_part[::-1]
    else:
        x_s_part = (np.arange(div/2)+1)*2*xs/div
        x_s[:int(div/2)] = x_s_part
        x_s[int(div/2):] = x_s_part[::-1]
        
    y_s = np.arange(div+1)*2*ys/div-ys
    
    for n in np.arange(div):
        x = np.array((-x_s[n],x_s[n]));    
        y = np.array((y_s[n],y_s[n+1]));      
        z = np.array((1738e3-td,1738e3-td-th))
    
        for i in np.arange(2):
            dxi = x[i]-xp;
            for j in np.arange(2):
                dyj = y[j]-yp;
                for k in np.arange(2):
                    mu = (-1)**i*(-1)**j*(-1)**k;
                    dzk = z[k]-zp;
                    R = (dxi**2 + dyj**2 + dzk**2)**0.5;
                    grav = grav + mu*(dzk*(np.arctan(dxi*dyj/(dzk*R)))
                        - dxi*np.log(R+dyj)
                        - dyj*np.log(R+dxi));
    
                
    
    grav = G*drho*grav;
    return grav

far_bg = random_coeff(mtsef_og)
bar = 1e10
xs_grid = []
th_grid = []
td_grid = []
bar_grid = []
lf = 500
start_time = time.time()

pass_num = 6

g0 = 1.6230976292682058

L = np.arange(0,lf+1)
taper0 = 1/((L+1)*g0)
taper = np.tile(taper0,(lf+1,1)).transpose()

taperf = np.zeros((2,lf+1,lf+1))
taperf[0,:,:] = taper
taperf[1,:,:] = taper

def likelihood(td,bd,xs,far_bg,mtseff0):
    
    grav_p = forward_grav(td,bd-td,xs/2,10)
    temp_grav = copy.deepcopy(grav_p)
    grav_p = copy.deepcopy(-temp_grav)
    tempp = pyshtools.expand.SHExpandDH(grav_p,lmax_calc=lf,sampling=2)
    tempp2 = tempp*taperf
    tempp2[0][0][0] = 1
    tempp2[:,1:pass_num,:] = 0
    specc, sdf = pyshtools.spectralanalysis.SHMultiTaperMaskSE(tempp2, tapersff0)
    # f0_syn = pyshtools.spectralanalysis.spectrum(tempp2 + far_bg)
    f0_syn = mtsef_og + specc
    summ = (sum((np.log(f0_syn[138:246])-np.log(mtseff0[138:246]))**2))/7.0984639568165635#uncertainty squared
    
    llh = np.exp(-summ**2/2)*np.exp(-(td-14e3)**2/(2*36e6))
 
    return llh


            
# import pdb 
# pdb.set_trace()
bd = int(sys.argv[1])
xs = int(sys.argv[2])
td = int(sys.argv[3])

bd_c = np.array([]);
td_c = np.array([]);
xs_c = np.array([]);


#number of steps for the MCMC
total = 1000

acc = 0
rej = 0


for step in np.arange(total):
    if step%10 == 0:
        elapsed_time = time.time() - start_time
        print(elapsed_time)
    llho = likelihood(td, bd, xs, far_bg, mtseff0)
    
    td1 = td+np.random.normal(0,3.5e3)
    bd1 = bd+np.random.normal(0,7e3)
    xs1 = bd+np.random.normal(0,7e3)
    llhn = likelihood(td1, bd1, xs1, far_bg, mtseff0)
    
    judge = np.random.uniform(0, 1)
    
    if td1 < 0 or bd1 > 50e3 or xs1 < 0 or td1 >= bd1:
        llhn = 0
        
    test = llhn/llho
    
    if test < judge:
        rej+=1
    else:
        acc+=1
        bd = copy.deepcopy(bd1)
        xs = copy.deepcopy(xs1)
        td = copy.deepcopy(td1)
        
        bd_c = np.append(bd_c,bd)
        xs_c = np.append(xs_c,xs)
        td_c = np.append(td_c,td)
        
    
   
filename = 'dike_results_mcmc' + '_' + sys.argv[1] + '_' + sys.argv[2] + '_' + sys.argv[3] + '.mat'
io.savemat(filename,{'bd_c':bd_c,'xs_c':xs_c,'td_c':td_c,'acc':acc,'rej':rej})


Sample Results (top labels are the initial conditions in the order of bottom depth, width, and top depth):

<img src="liang_figures/top_depth_results.png" style="width:45%">

<img src="liang_figures/bottom_depth_results.png" style="width:45%">

<img src="liang_figures/width_results.png" style="width:45%">

